In [118]:
import requests
import html
import urllib.request
import urllib.error
import urllib.parse
from bs4 import BeautifulSoup as bs

import time
import os
import seaborn as sns
import pandas as pd
import scipy as sc
import numpy as np
import re

import statsmodels.formula.api as sm

import matplotlib.pyplot as plt 
plt.style.use('ggplot')
%matplotlib inline  
plt.rcParams['figure.figsize'] = (10, 6) 

## Important Note:<br>
## If you want to build the DataFrame by your own and download the relevant files to achive this task then begin from here.<br> 
## If you want to run the code when you already using existed DataFrame and downloaded files, Please skip to Phase 2 - Learning Algorithm & Performance Evaluation

# Phase 1 - Data Acquisition & Data Cleaning & Data Vectorization

# Part 1 - Load and clean the data
The following cells perform 2 things:
* load the csv file which contain the dataframe of spotify songs
* replace all of the unnecessary punctuation in each title and artist for the further work with scraping

In [119]:
df_spotify=pd.read_csv("./data/spotify.csv")
df_spotify=df_spotify.drop('Unnamed: 0', 1)    

## Define a function to clean the titles and artists string for the search in metrolyrics

In [120]:
def cleanDf(df):
    for i in range(0,len(df)):
        df['Title'][i]=df['Title'][i].replace('MotorSport','motor sport').replace('PILLOWTALK','pillow talk').replace('Back To You','Back 2 you')
        
        list=df['Title'][i].split(' (feat. ')
        df['Title'][i]=list[0]
        list=df['Title'][i].split('- From ')
        df['Title'][i]=list[0]
        list=df['Title'][i].split(' feat. ')
        df['Title'][i]=list[0]
        list=df['Title'][i].split(' (with ')
        df['Title'][i]=list[0]
        list=df['Title'][i].split(' (Original ')
        df['Title'][i]=list[0]
        list=df['Title'][i].split(' (From ')
        df['Title'][i]=list[0]
        list=df['Title'][i].split(' (Fifty ')
        df['Title'][i]=list[0]
        
        df['Title'][i]=df['Title'][i].replace("Wanna","want to").replace('\n','').replace(';','').replace("'","").replace(',','').replace('/ ','').replace('- ','').replace('-','').replace('é','e').replace('?','').replace('"','').replace("!","").replace("in'","ing").replace(" and "," ").replace("’","")
        df['Title'][i]=df['Title'][i].replace('(','').replace(')','').replace('.','')
        
        df['Artist'][i]=df['Artist'][i].replace('P!nk','Pink').replace('NERD','nerd the neptunes').replace('ZAYN','zayn malik').replace('Axwell /\ Ingrosso','Axwell Ingrosso').replace('Ayo & Teo','Ayo Teo')
        df['Artist'][i]=df['Artist'][i].replace('é','e').replace('í','i').replace('.','').replace('\n','').replace(' + ','-').replace('.','').replace('ó','o').replace('$','s').replace("'","").replace("!","")

## Clean our dataframe

In [121]:
cleanDf(df_spotify)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to

In [122]:
df_spotify[:100]

,Title,Artist,id,artist_genres,Year,is_top100,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Love Yourself,Justin Bieber,3hB5DgAiMAQ4DzYbsMq1IT,"['canadian pop', 'pop', 'post-teen pop']",2016,1,0.83500,0.609,233720,0.3780,0.000000,4,0.2800,-9.828,1,0.4380,100.418,4,0.5150
1,Sorry,Justin Bieber,69bp2EbF7Q2rqc5N3ylezZ,"['canadian pop', 'pop', 'post-teen pop']",2016,1,0.07970,0.654,200787,0.7600,0.000000,0,0.2990,-3.669,0,0.0450,99.945,4,0.4100
2,One Dance,Drake,1xznGGDReH1oQq0xzbwXa3,"['canadian hip hop', 'canadian pop', 'hip hop'...",2016,1,0.00784,0.791,173987,0.6190,0.004230,1,0.3510,-5.886,1,0.0532,103.989,4,0.3710
3,Work,Rihanna,14WWzenpaEgQZlqPq2nk4v,"['barbadian pop', 'dance pop', 'pop', 'post-te...",2016,1,0.07520,0.725,219320,0.5340,0.000000,11,0.0919,-6.238,1,0.0946,91.974,4,0.5580
4,Stressed Out,Twenty One Pilots,3CRDbSIZ4r5MsZ0YwxuEkn,"['modern rock', 'rock']",2016,1,0.04620,0.734,202333,0.6370,0.000023,4,0.0602,-5.677,0,0.1410,169.977,4,0.6480
5,Panda,Desiigner,5OOkp4U9P9oL23maHFHL1h,"['pop rap', 'rap', 'southern hip hop', 'trap',...",2016,1,0.02800,0.576,246761,0.7660,0.000002,10,0.3660,-4.943,0,0.4490,144.833,4,0.2360
6,Hello,Adele,0ENSn4fwAbCGeFGVUbXEU3,"['british soul', 'pop', 'uk pop']",2016,1,0.33600,0.481,295493,0.4510,0.000000,5,0.0872,-6.095,0,0.0347,157.966,4,0.2890
7,Dont Let Me Down,The Chainsmokers,1i1fxkWeaMmKEB4T7zqbzK,"['electropop', 'pop', 'tropical house']",2016,1,0.15700,0.532,208373,0.8690,0.005080,11,0.1360,-5.094,1,0.1720,159.803,4,0.4220
8,CANT STOP THE FEELING,Justin Timberlake,6JV2JOEocMgcZxYSZelKcc,"['dance pop', 'pop']",2016,1,0.01230,0.667,236002,0.8300,0.000000,0,0.1910,-5.715,1,0.0749,113.030,4,0.7010
9,Closer,The Chainsmokers,7BKLCZ1jbUBVqRi2FVlTVw,"['electropop', 'pop', 'tropical house']",2016,1,0.41400,0.748,244960,0.5240,0.000000,8,0.1110,-5.599,1,0.0338,95.010,4,0.6610


## Checking the amount of rows in dataframe

In [123]:
len(df_spotify)

586

# Part 2 - Load and store lyrics data

* clean the text from unnecessary punctuation

## Define a function to create the lyrics_urls list to store each song's lyrics url on metrolyrics
The following function perform 2 things:
* create list of urls from metrolyrics for each song 
* replace all of the unnecessary strings from the title of each song
* handle with cases of several artists of a song. split them and take the first one to build with it a url
* replace all of the spaces to '-' for the url
* clean the text from unnecessary punctuation

In [124]:

def create_lyrics_urls(df):
    #creating the list of urls for each song's lyrics
    lyrics_urls=[]
    for i in range (0,len(df)):
        #the case of & in songs:
        name=df['Title'][i].replace("& ","").replace("'","")
        name=name.replace(" ","-")

        list=df['Artist'][i].split(',')
        #we will always take the first artist
        artist_str=list[0]
        list=artist_str.split(' & ')
        artist_str=list[0]
        list=artist_str.split(' and ')
        artist_str=list[0]
        list=artist_str.split('featuring')
        artist_str=list[0]

        url='https://www.metrolyrics.com/'+name+'-lyrics-'+artist_str.replace('"','').replace(" ","-")+'.html'
        #the case of "Maroon-5-.html"
        url=url.replace("-.",".") 

        lyrics_urls.append(url)      
    return lyrics_urls    

In [125]:
urls=create_lyrics_urls(df_spotify)
len(urls) 

586

### Loading the nltk package for the further work with text

In [126]:
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords, movie_reviews

## Define function to delete the stopwords from the lyrics

In [127]:
stop_words = set(stopwords.words('english'))

def remove_mystopwords(string):
    tokens = string.split(" ")
    tokens_filtered = [word for word in tokens if not word in stop_words]
    return (" ").join(tokens_filtered)

## The following cell pefrom 4 things:
* scrap the metrolyrics
* load the lyrics of each song
* clean the text from digits, stopwords, and unnecessary punctuation
* check the amount of words in each song 

In [128]:
#the part of creation of the lyrics_texts dictionary which contain the lyrics of each song in top 2016,2017,2018

all_words_count=[]

def find_lyrics_txt(df):
    lyrics_texts=[]
    time.sleep(7)
    for i in range (0,len(df)):
        url = urls[i]
        response = requests.get(url)
        data = response.text
        soup = bs(data,'html.parser')
        couplets = soup.findAll("p",{"class":"verse"})
        temp = ""
        for p in couplets:
            temp = str(temp) + str(p.text)
            temp=temp+"\n"
        #cleaning the text from unnecessary punctuation
        temp=re.sub(r'[^(a-zA-Z)\s]','', temp)
        temp=temp.replace('[','').replace(']','').replace('(','').replace(')','').replace('"','').replace("'","")
        #remove all of the digits 
        temp=''.join(j for j in temp if not j.isdigit())
        #count the number of words in each song
        all_words_count.append(len(temp))
        #remove all of the stopwords
        temp = remove_mystopwords(temp)
        #for the further work with text we would like to lowercase all of the words in text   
        lyrics_texts.append(temp.lower())
        lyrics_texts[i]=lyrics_texts[i].replace('\n ','\n')  
    return lyrics_texts    

In [129]:
lyrics=find_lyrics_txt(df_spotify)

In [130]:
lyrics_without_empty_strings = []
for string in lyrics:
    if (string != ""):
        lyrics_without_empty_strings.append(string)

In [131]:
len(all_words_count)

586

## In the next cell we want to check the amount of songs with zero words (the algorithm of finding the url for metrolyrics didn't work for them) so we can delete them in the dataframe in the future

In [132]:
res_list = [] 
count=0
for i in range(0, len(all_words_count)) : 
    if all_words_count[i] == 0 : 
        res_list.append(i)    
print(res_list)        

[120, 123, 135, 162, 241, 246, 279, 298, 300, 301, 302, 308, 311, 315, 316, 317, 318, 319, 322, 329, 333, 338, 341, 352, 364, 376, 378, 383, 387, 388, 390, 392, 397, 404, 406, 410, 414, 415, 417, 430, 432, 435, 442, 444, 445, 451, 453, 455, 458, 466, 473, 476, 480, 481, 482, 483, 486, 487, 498, 502, 510, 519, 521, 523, 531, 532, 537, 540, 544, 554, 561, 563, 568, 579]


## In the next cell we create a function to add a column to our dataframe

In [133]:
def append_col(df,lst,colName):
    df[colName] = lst

## Add to our dataframe the 'words_count' column

In [134]:
append_col(df_spotify,all_words_count,'words_count')

## Cleaning: In the next cell we drop every row which value of amout of words in song is 0

In [135]:

for i in range (0,len(df_spotify)):
    if (df_spotify['words_count'][i]) == 0:
        df_spotify['words_count'][i]=pd.np.NaN
        
df_spotify=df_spotify.dropna()

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


## Check the amount of rows in a new dataframe without nans

In [136]:
len(df_spotify)

512

## Part 3 - Work with lyrics text
The following cells pefrom 3 things:
* create a list of negative words and find how many times do they appear in each song
* create a list of positive words and find how many times do they appear in each song
* find a total count of repeated words

# Defining a text as positive or negative using textblob package

In [137]:
!pip install textblob

## Getting the polarity value for each song

In [138]:
from textblob import TextBlob
# The sentiment function of textblob returns two properties, polarity, and subjectivity.
# Polarity is float which lies in the range of [-1,1] where 1 means positive statement and -1 means a negative statement

def pos_neg_neutral(lyrics_texts):
    blobs=[]
    for i in range (0,len(lyrics_texts)):
        blob=TextBlob(lyrics_texts[i])
        pnn=blob.sentiment.polarity
        definition=''
        if -0.3 < pnn < 0.3:
            definition='neutral'
        if pnn  <= -0.3:
            definition='negative'
        if pnn >= 0.3:
            definition='positive'
        blobs.append(definition)    
    return blobs

In [139]:
blobs_polarity=pos_neg_neutral(lyrics_without_empty_strings)

In [140]:
len(blobs_polarity)

512

In [141]:
print(blobs_polarity)

['neutral', 'negative', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'positive', 'neutral', 'neutral', 'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'positive', 'positive', 'neutral', 'positive', 'neutral', 'neutral', 'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'positive', 'neutral', 'neutral', 'negative', 'positive', 'neutral', 'neutral', 'neutral', 'neutr

## In the next cell we will check hom many negative,neutral,positive songs do we have

In [142]:
blobs_res={"pos": 0, "neg": 0, "neutral": 0}
for i in range (0,len(blobs_polarity)):
    if blobs_polarity[i]=="positive":
        blobs_res["pos"]+=1
    if blobs_polarity[i]=="negative":
        blobs_res["neg"]+=1
    if blobs_polarity[i]=="neutral":
        blobs_res["neutral"]+=1
print(blobs_res)        

{'pos': 50, 'neg': 7, 'neutral': 455}


In [143]:
append_col(df_spotify,blobs_polarity,'blobs_polarity')

## In the next cell we will tokenize (convert the whole lyrics text to words) the lyrics of each song

In [144]:
tokenized_lyrics=[]

for i in range (0,len(df_spotify)):
    tokens=word_tokenize(lyrics_without_empty_strings[i])
    tokenized_lyrics.append(tokens)

In [145]:
print(str(tokenized_lyrics[0]))

['for', 'times', 'rain', 'parade', 'and', 'clubs', 'get', 'using', 'name', 'you', 'think', 'broke', 'heart', 'oh', 'girl', 'goodness', 'sake', 'you', 'think', 'im', 'crying', 'well', 'i', 'aint', 'and', 'i', 'didnt', 'wan', 'na', 'write', 'song', 'cause', 'i', 'didnt', 'want', 'anyone', 'thinking', 'i', 'still', 'care', 'i', 'dont', 'still', 'hit', 'phone', 'up', 'and', 'baby', 'i', 'movin', 'on', 'and', 'i', 'think', 'somethin', 'i', 'dont', 'wan', 'na', 'hold', 'back', 'maybe', 'know', 'that', 'my', 'mama', 'dont', 'like', 'likes', 'everyone', 'and', 'i', 'never', 'like', 'admit', 'i', 'wrong', 'and', 'ive', 'caught', 'job', 'didnt', 'see', 'whats', 'going', 'on', 'but', 'i', 'know', 'im', 'better', 'sleeping', 'own', 'cause', 'like', 'way', 'look', 'much', 'oh', 'baby', 'go', 'love', 'yourself', 'and', 'think', 'im', 'still', 'holdin', 'somethin', 'you', 'go', 'love', 'yourself', 'and', 'told', 'hated', 'friends', 'the', 'problem', 'them', 'and', 'every', 'time', 'told', 'opinion', 

In [146]:
#here we would like to count how many positive words can we see in each song lyrics

count_pos=[]

keywords_pos=[]

url='https://www.enchantedlearning.com/wordlist/positivewords.shtml'
response = requests.get(url)
data = response.text
soup = bs(data,'html.parser')
words = soup.findAll("div",{"class":"wordlist-section"})

for div in words:
    divs=soup.findAll("div",{"class":"wordlist-item"})
    for div in divs:
        keywords_pos.append(div.text)

for i in range (0,len(df_spotify)):
    temp=[]
    for word in keywords_pos:
        #here we create a temp list to store all the words from keywords_neg which appear in lyrics 
        temp+=re.findall(word, str(lyrics_without_empty_strings[i]))
    #now we'd want to know how many times these words appeared in text
    count_pos.append(len(temp))
print(count_pos)         

[350, 600, 525, 450, 525, 200, 200, 400, 850, 425, 375, 300, 275, 825, 675, 175, 150, 450, 800, 200, 400, 875, 150, 900, 225, 950, 300, 325, 575, 550, 275, 175, 675, 350, 450, 200, 300, 350, 375, 425, 200, 425, 775, 125, 325, 275, 175, 325, 300, 575, 400, 1050, 275, 625, 200, 175, 1875, 450, 1025, 325, 275, 75, 250, 450, 275, 350, 100, 400, 225, 725, 325, 250, 1050, 525, 475, 275, 1000, 175, 525, 375, 175, 950, 150, 2375, 175, 1250, 625, 375, 275, 425, 125, 675, 100, 500, 275, 525, 200, 800, 125, 50, 675, 475, 550, 700, 425, 275, 500, 825, 375, 2450, 325, 200, 650, 600, 200, 275, 475, 450, 25, 425, 725, 375, 200, 400, 325, 650, 575, 350, 125, 575, 725, 925, 425, 225, 275, 250, 250, 275, 600, 450, 175, 575, 700, 850, 0, 400, 225, 1275, 450, 500, 250, 400, 1325, 625, 450, 475, 500, 625, 175, 300, 300, 275, 350, 375, 375, 650, 675, 225, 350, 1250, 175, 0, 525, 225, 675, 475, 400, 275, 875, 450, 175, 125, 175, 300, 325, 250, 325, 425, 125, 400, 350, 0, 175, 175, 525, 400, 125, 250, 250, 27

In [147]:
#here we would like to count how many negative words can we see in each song lyrics

count_neg=[]

keywords_neg=[]

url='https://www.enchantedlearning.com/wordlist/negativewords.shtml'
response = requests.get(url)
data = response.text
soup = bs(data,'html.parser')
words = soup.findAll("div",{"class":"wordlist-section"})

for div in words:
    divs=soup.findAll("div",{"class":"wordlist-item"})
    for div in divs:
        keywords_neg.append(div.text)
        
for i in range (0,len(df_spotify)):
    temp=[]
    for word in keywords_neg:
        #here we create a temp list to store all the words from keywords_neg which appear in lyrics 
        temp+=re.findall(word, str(lyrics_without_empty_strings[i]))
    #now we'd want to know how many times these words appeared in text
    count_neg.append(len(temp))
print(count_neg)  

[648, 1104, 384, 816, 336, 600, 312, 288, 480, 840, 552, 1032, 504, 456, 1008, 240, 72, 624, 1032, 336, 264, 672, 672, 600, 600, 360, 456, 360, 288, 552, 960, 168, 336, 312, 456, 288, 456, 1272, 456, 1320, 408, 672, 576, 336, 960, 600, 624, 672, 240, 456, 216, 1656, 360, 192, 216, 288, 312, 360, 672, 552, 408, 192, 168, 408, 360, 1104, 576, 1056, 240, 312, 504, 168, 960, 504, 456, 480, 1056, 96, 600, 432, 456, 120, 168, 2976, 168, 1008, 840, 648, 216, 336, 168, 528, 72, 168, 2064, 1080, 432, 432, 192, 360, 288, 408, 120, 1968, 840, 240, 456, 792, 552, 840, 672, 408, 1152, 288, 144, 432, 1032, 360, 408, 336, 672, 264, 312, 816, 360, 600, 624, 480, 288, 288, 624, 576, 144, 912, 312, 768, 96, 480, 1152, 720, 624, 1488, 192, 480, 192, 120, 576, 504, 216, 384, 312, 264, 1296, 888, 336, 576, 456, 552, 432, 312, 168, 1728, 312, 216, 240, 384, 864, 288, 144, 1008, 144, 360, 648, 168, 528, 288, 288, 480, 624, 576, 168, 240, 96, 168, 360, 264, 384, 672, 96, 528, 984, 0, 1608, 504, 336, 624, 96, 

In [148]:
count_swear=[]

keywords_swear=[]

url='https://en.wiktionary.org/wiki/Category:English_swear_words'
response = requests.get(url)
data = response.text
soup = bs(data,'html.parser')
div_class = soup.findAll("div",{"class":"mw-category-group"})

for div in div_class:
    uls=div.find_all('ul')
    for ul in uls:
        lis=ul.find_all('li')
        for ls in lis:
            links=ls.find_all('a')
            for link in links:
                keywords_swear.append(link.text)
                
keywords_swear.pop(0)         

for k in range(0,len(keywords_swear)):
    keywords_swear[k] =keywords_swear[k].lower() 

# print(keywords_swear)

for i in range (0,len(df_spotify)):
    temp=[]
    for word in keywords_swear:
        #here we create a temp list to store all the words from keywords_neg which appear in lyrics 
        temp+=re.findall(word, str(lyrics_without_empty_strings[i]))
    #now we'd want to know how many times these words appeared in text
    count_swear.append(len(temp))
print(count_swear)  

[0, 0, 0, 1, 0, 24, 9, 0, 0, 0, 0, 0, 14, 2, 1, 0, 0, 8, 8, 0, 0, 3, 0, 2, 0, 0, 0, 0, 0, 28, 0, 17, 0, 25, 6, 0, 4, 10, 0, 3, 0, 0, 0, 0, 0, 0, 2, 3, 0, 0, 0, 0, 5, 12, 1, 0, 11, 13, 6, 16, 6, 3, 0, 0, 11, 6, 0, 1, 5, 1, 4, 0, 7, 11, 0, 2, 0, 1, 0, 10, 0, 0, 0, 0, 1, 0, 10, 10, 16, 6, 2, 3, 0, 0, 11, 41, 1, 0, 0, 1, 1, 51, 0, 50, 0, 1, 1, 8, 0, 5, 2, 19, 7, 4, 1, 3, 6, 13, 0, 0, 22, 8, 0, 8, 1, 0, 1, 0, 3, 0, 23, 0, 1, 0, 1, 1, 1, 2, 0, 0, 2, 7, 25, 0, 0, 2, 23, 70, 0, 10, 13, 0, 0, 21, 1, 13, 0, 16, 20, 2, 4, 1, 25, 4, 1, 3, 3, 6, 0, 0, 0, 20, 9, 23, 3, 7, 0, 4, 21, 0, 0, 0, 0, 0, 0, 18, 2, 27, 1, 0, 16, 25, 0, 0, 3, 0, 1, 13, 7, 8, 0, 16, 1, 10, 2, 1, 3, 0, 1, 1, 0, 3, 0, 2, 2, 28, 2, 6, 0, 1, 23, 1, 44, 15, 0, 0, 14, 3, 1, 0, 20, 3, 4, 0, 17, 7, 27, 0, 4, 3, 0, 1, 24, 0, 22, 2, 46, 21, 0, 16, 0, 44, 0, 21, 7, 0, 0, 0, 1, 1, 0, 0, 18, 4, 15, 4, 0, 0, 0, 20, 0, 1, 1, 1, 0, 6, 0, 18, 0, 0, 5, 0, 0, 5, 1, 0, 30, 9, 0, 1, 0, 0, 3, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 11, 8, 0, 2, 1, 0, 6, 0, 0

In [149]:
append_col(df_spotify,count_neg,'words_neg_count')
append_col(df_spotify,count_pos,'words_pos_count')
append_col(df_spotify,count_swear,'words_curse_count')

## In the next cell we'd like to find the total count of repeated words and the most common words in each song

In [150]:
from collections import Counter

reg = re.compile('\S{4,}')

repeated_words=[]
most_common_words=[]

for i in range (0,len(df_spotify)):
    c = Counter(ma.group() for ma in reg.finditer(lyrics_without_empty_strings[i]))
    repeated_words.append(sum(c.values()))
    most_common_words.append([k for k,v in c.most_common()])

In [151]:
print(repeated_words)

[179, 117, 184, 307, 265, 362, 142, 150, 252, 196, 173, 187, 118, 167, 200, 234, 60, 215, 314, 159, 163, 142, 171, 221, 103, 171, 88, 109, 208, 439, 182, 203, 182, 258, 166, 228, 252, 229, 187, 133, 195, 156, 93, 111, 170, 159, 120, 259, 151, 216, 204, 289, 152, 221, 135, 149, 347, 219, 164, 179, 163, 106, 129, 136, 198, 172, 99, 266, 112, 175, 122, 158, 210, 228, 199, 204, 269, 88, 183, 185, 165, 174, 122, 351, 30, 151, 204, 285, 178, 310, 179, 134, 110, 157, 205, 347, 146, 222, 267, 329, 212, 215, 169, 552, 196, 144, 185, 240, 149, 351, 200, 202, 211, 221, 124, 214, 319, 219, 143, 210, 296, 116, 153, 295, 120, 206, 243, 135, 151, 257, 348, 312, 215, 187, 153, 221, 154, 204, 310, 126, 120, 232, 285, 252, 125, 149, 323, 351, 157, 277, 223, 163, 180, 250, 249, 317, 142, 205, 107, 121, 271, 277, 258, 218, 158, 238, 216, 73, 151, 150, 168, 223, 245, 251, 134, 177, 117, 242, 403, 162, 159, 108, 168, 94, 109, 273, 174, 218, 190, 125, 159, 111, 157, 151, 127, 144, 190, 223, 269, 422, 160, 20

In [152]:
print(most_common_words[0])

['love', 'think', 'still', 'like', 'yourself', 'didnt', 'cause', 'dont', 'baby', 'somethin', 'wanna', 'know', 'look', 'much', 'holdin', 'never', 'wrong', 'times', 'write', 'song', 'want', 'anyone', 'thinking', 'care', 'phone', 'movin', 'hold', 'back', 'maybe', 'that', 'mama', 'likes', 'everyone', 'admit', 'caught', 'whats', 'going', 'better', 'sleeping', 'told', 'feel', 'rain', 'parade', 'clubs', 'using', 'name', 'broke', 'heart', 'girl', 'goodness', 'sake', 'crying', 'well', 'aint', 'hated', 'friends', 'problem', 'them', 'every', 'time', 'opinion', 'tried', 'make', 'forget', 'came', 'from', 'made', 'small', 'fell', 'nothin', 'felt', 'vulnerable', 'fool', 'break', 'walls']


In [153]:
append_col(df_spotify,repeated_words,'words_repeat_count')
append_col(df_spotify,most_common_words,'common_words')

In [163]:
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [155]:
print(df_spotify.shape)
df_spotify.dtypes

(512, 26)


Title                  object
Artist                 object
id                     object
artist_genres          object
Year                    int64
is_top100               int64
acousticness          float64
danceability          float64
duration_ms             int64
energy                float64
instrumentalness      float64
key                     int64
liveness              float64
loudness              float64
mode                    int64
speechiness           float64
tempo                 float64
time_signature          int64
valence               float64
words_count           float64
blobs_polarity         object
words_neg_count         int64
words_pos_count         int64
words_curse_count       int64
words_repeat_count      int64
common_words           object
dtype: object

In [158]:
#df_spotify['common_words']

In [159]:
y = df_spotify['is_top100'].values.tolist()
type(y)

list

<br>**Next step:**
* Remove non relevant column for predictions from our dataframe.
* Change string columns 

In [160]:
print(df_spotify.shape)
df = df_spotify.drop(['Title', 'Artist', 'id', 'is_top100'], 1)
print(df.shape)

(512, 26)
(512, 22)


In [167]:
le = preprocessing.LabelEncoder()
le.fit(df['artist_genres'])
list(le.classes_)

["['acoustic pop']",
 "['alternative metal', 'christian alternative rock', 'christian music', 'christian rock', 'post-grunge']",
 "['alternative pop rock', 'pop']",
 "['alternative r&b', 'atl hip hop', 'hip hop', 'melodic rap', 'pop', 'pop rap', 'rap', 'trap']",
 "['alternative r&b', 'hip hop', 'lgbtq+ hip hop', 'neo soul', 'pop']",
 "['alternative r&b', 'indie soul']",
 "['alternative r&b', 'melodic rap', 'pop rap']",
 "['alternative r&b', 'pop']",
 "['alternative r&b', 'viral pop']",
 "['alternative rock', 'funk metal', 'funk rock', 'permanent wave', 'post-grunge', 'rock']",
 "['atl hip hop', 'atl trap', 'gangster rap', 'melodic rap', 'pop rap', 'rap', 'trap']",
 "['atl hip hop', 'atl trap', 'melodic rap', 'rap', 'southern hip hop', 'trap']",
 "['atl hip hop', 'atl trap', 'rap']",
 "['atl hip hop', 'dance pop', 'pop', 'r&b', 'urban contemporary']",
 "['atl hip hop', 'dirty south rap', 'hip hop', 'pop rap', 'rap', 'southern hip hop', 'trap']",
 "['atl hip hop', 'hip hop', 'melodic rap

In [165]:
le.transform(df['artist_genres'])

array([ 57,  57,  54,  27, 180, 187,  44, 129, 114, 129,  24, 115,  27,
        87, 105,  98,  80, 106, 145, 180, 180,  95,  56,  54,  81,  44,
       129,  56,  54,  20,  57,  51, 112, 188, 193, 112,  54, 204,  50,
       112,  54,  99, 111, 182,  99,  88,  83, 178,  73, 111, 112,  94,
       193, 184,  59, 112,  31,  51,  78,  23,  51,  99, 211,  74, 119,
       111, 179, 147, 120,  55,  51, 185,  55, 207,  56, 112,  84,  83,
        86,  54,  44,  84,  81,  56,  45, 186, 103,  13, 143,  31,  98,
       112, 185,  75,  20, 187, 149,  41, 202, 169, 200,  70, 129,  17,
       129,  71, 180, 119, 197,  78, 173,  20, 153, 200,  66,  78, 146,
        51,   7, 111, 194,  18, 124, 194,  96,  50,  67, 112,  27,  51,
       118,  90,  54, 112, 202, 189, 129, 112,  89, 159,  83, 112,  21,
       114, 171, 180,  17, 120,  56, 139, 119, 180,  93, 151, 209,  70,
       211, 146,  54,  96, 207, 196, 188,  80,  72, 112,  80, 190,  77,
       198,   7,  19, 194, 174, 112,  14, 112, 102,  17, 200,  4

In [169]:
list(le.inverse_transform([36]))

["['big room', 'dance pop', 'edm', 'electro house', 'pop', 'progressive electro house', 'progressive house', 'tropical house']"]

# Phase 2 - Learning Algoorithm & Performance Evaluation

In [161]:
XTrain, XTest, yTrain, yTest = train_test_split(df, y, test_size=0.2)

In [162]:
# clf = tree.DecisionTreeClassifier()
# clf.fit(XTrain, yTrain)

ValueError: could not convert string to float: "['canadian contemporary r&b', 'canadian pop', 'pop']"